<a href="https://colab.research.google.com/github/seholton/Amazon_zine_Analysis/blob/main/Amazon_Vine_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
tar: spark-3.2.1-bin-hadoop2.7.tgz: Cannot open: No

In [11]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-21 19:46:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.76MB/s    in 0.2s    

2022-09-21 19:46:02 (5.76 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreat

ModuleNotFoundError: ignored

In [ ]:
## Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

In [ ]:
## Create DataFrame matching Tables

In [ ]:
from pyspark.sql.functions import col
# Read in the Review dataset as a DataFrame
df.show(20)

ModuleNotFoundError: ignored

In [ ]:
totalvotes20_df = df.filter(col("total_votes") >= 20)
totalvotes20_df.show(20)

In [ ]:
totalvotes20_df.count

In [ ]:
helpfulvotes50_df = totalvotes20_df.filter(col("helpful_votes")/col("total_votes") >= .50)
helpfulvotes50_df.show(5)

In [ ]:
helpfulvotes50_df.count()

In [ ]:
paidvine_df = helpfulvotes50_df.filter(col("vine") == 'Y')
paidvine_df.show(5)

In [ ]:
paidvine_df.count()

In [ ]:
unpaidvine_df = helpfulvotes50_df.filter(col("vine") == 'N')
unpaidvine_df.show(5)

In [ ]:
unpaidvine_df.count()

In [ ]:
paidrating_df = paidvine_df.filter(col("star_rating") == 5)
paidrating_df.count()

In [ ]:
unpaidrating_df = unpaidvine_df.filter(col("star_rating") == 5)
unpaidrating_df.count()

In [ ]:

percent_paid = (paidrating_df.count()/paidvine_df.count())*100
print("Percentage of 5-star reviews for Paid Vine program: %f" % percent_paid + "%")

In [ ]:
percent_unpaid = (unpaidrating_df.count()/unpaidvine_df.count())*100
print("Percentage of 5-star reviews for Unpaid Vine program: %f" % percent_unpaid + "%")